In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import time
import json
import re

## Setup - setting your driver path

In [2]:
import platform
if platform.system() == "Windows":
    driverpath = os.getcwd() + "/chromedriver.exe"
else:
    driverpath = os.getcwd() + "/chromedriver"


options = webdriver.ChromeOptions()
#options.add_argument("headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])

In [3]:
driver = webdriver.Chrome(driverpath,  chrome_options=options)

/var/folders/lk/b7zqk0ps4792145cxptl4zwc0000gn/T/ipykernel_26031/2209922887.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driverpath,  chrome_options=options)


## Move to URL

In [4]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
#Error Case
url = "https://portal.yonsei.ac.kr/main/"
driver.get(url)
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "RANDOMID")))
#time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser") 

In [7]:
url = "https://portal.yonsei.ac.kr/main/"
driver.get(url)
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "icon_menu")))
#time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser") 

## Interact with web objects (Make screen half sized)

In [8]:
from selenium.webdriver.common.action_chains import ActionChains

In [10]:
driver.find_element_by_class_name("icon_menu").click()

In [17]:
element = driver.find_element_by_class_name("icon_menu")

In [19]:
element.click()

## Maximized window

In [8]:
driver.maximize_window()
html = driver.page_source
soup = BeautifulSoup(html, "html.parser") 

In [24]:
chunk = soup.find("ul", {"class" : "sub double"})
menus = [i.text for i in chunk.find_all("li")]

In [25]:
menus

['학사관리',
 '학술정보원 (도서관)',
 '고위자과정',
 '학생증 발급',
 '전자출석부',
 '상담센터',
 '온라인강의(LearnUS)',
 '리더십센터',
 '취업안내',
 '교수학습혁신센터',
 '각종 신청서양식 모음',
 '공익법률지원센터']

## Send keys (login, etc)

In [37]:
login_url = "https://accounts.google.com/signin"
driver.get(login_url)

In [ ]:
ID = "youid@id.com"
PW = "yourpw"
def login_get_main():
    print ("Signing in...")
    driver.get("https://accounts.google.com/signin")
    elem = driver.find_element_by_xpath('//*[@id="identifierId"]')
    elem.send_keys(ID)
    driver.find_element_by_xpath('//*[@id="identifierNext"]').click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input'))
    )
    elem = driver.find_element_by_xpath('//*[@id="password"]/div[1]/div/div[1]/input')
    elem.send_keys(PW)
    driver.find_element_by_xpath('//*[@id="passwordNext"]').click()

    time.sleep(5)
    print ("Sined in! getting to main...")
    driver.get(main)

## Example of Appstore Crawling

In [13]:
def scroll(how_long=100):
    SCROLL_PAUSE_TIME = 2
    # Get scroll height
    starttime = time.time()
    proceed = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    while proceed<how_long:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        endtime = time.time()
        proceed = endtime-starttime

In [11]:
#driver = webdriver.Chrome(driverpath,  chrome_options=options)
def get_rating(url, seconds = 100):
    driver.get(url)
    driver.implicitly_wait(5)
    scroll(seconds)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser") 
    reviews = soup.find_all("div", {"class" :  "we-customer-review lockup"})
    collection = {}
    for i in range(0, len(reviews)):
        collection[i] = {}
        l = reviews[i]
        collection[i]["title"] = l.find("h3", {"class" : "we-truncate we-truncate--single-line we-customer-review__title"}).text.strip()
        collection[i]["contents"] = l.find("div", {"class" : "we-clamp"}).text
        collection[i]["rating"] = l.find("figure", {"class" : "we-star-rating we-customer-review__rating we-star-rating--large"})["aria-label"]
        collection[i]["time"] = l.find("time")["datetime"]

    return collection

In [9]:
link = "https://apps.apple.com/us/app/keynote/id409183694?mt=12#see-all/reviews"

In [ ]:
get_rating(link, seconds = 5)